# Init

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

In [2]:
from scipy import interpolate
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [20]:
import parula as par

In [29]:
import toolsdiadem.tools as dia

In [3]:
root_dir = "/home/pleroy/DATA/DIADEM/bfu"
# 011 ===== 3l +45
dir_011 = "/20181015/011_3l_05l_+45"
pre_011 = "ey"
dat_011 = root_dir + dir_011 + "/" + pre_011
# 012 ===== 3l -45
dir_012 = "/20181015/012_3l_05l_-45"
pre_012 = "ey"
dat_012 = root_dir + dir_012 + "/" + pre_012
# 013_0 ===== 7l +45
dir_013_0 = "/20181016/013_seq/SEQ_0_7l_+45"
pre_013_0 = "ey"
dat_013_0 = root_dir + dir_013_0 + "/" + pre_013_0
# 013_1 ===== 7l -45
dir_013_1 = "/20181016/013_seq/SEQ_1_7l_-45"
pre_013_1 = "ey"
dat_013_1 = root_dir + dir_013_1 + "/" + pre_013_1
# 014_0 ===== 5l +45
dir_014_0 = "/20181017/014_5l_05l/SEQ_0_5l_+45"
pre_014_0 = "ey"
dat_014_0 = root_dir + dir_014_0 + "/" + pre_014_0
# 014_1 ===== 5l +-5
dir_014_1 = "/20181017/014_5l_05l/SEQ_1_5l_-45"
pre_014_1 = "ey"
dat_014_1 = root_dir + dir_014_1 + "/" + pre_014_1

In [4]:
import os
import fnmatch

In [5]:
x0y0 = (211, 244.5)

# Get frequencies and y values

## Get Nf and Ny

In [6]:
def get_Nf_Ny(data_dir):
    f_y = []
    for file in os.listdir(data_dir):
        if fnmatch.fnmatch(file, "*.amp"):
            sfile = file.split(".amp")[0].split("_")
            f_y.append( (sfile[1], sfile[2]) )
    Nf = max( [int(f_y[0]) for f_y in f_y if f_y[1] == '000'] ) + 1
    Ny = max( [int(f_y[1]) for f_y in f_y if f_y[0] == '000'] ) + 1
    print(data_dir)
    print("Nf = {}, Ny = {}".format(Nf, Ny))
    return Nf, Ny

In [7]:
Nf, Ny = get_Nf_Ny(root_dir + dir_011)

/home/pleroy/DATA/DIADEM/bfu/20181015/011_3l_05l_+45
Nf = 9, Ny = 81


## Get f and y

In [8]:
def getF(nbFreq, data_prefix):
    f = np.zeros(nbFreq)
    for freqIdx in range( nbFreq ):
        filename = data_prefix + f"_{freqIdx:03d}_000.amp"
        with open(filename) as file:
            for line in file:
                if "FreqValue" in line:
                    f[freqIdx] = float(line.split("=")[1])
                elif "Data#1" in line:
                    break
    print(root_dir + " Nf = {}, fmin = {}, fmax = {}".format(f.size, np.amin(f), np.amax(f)))
    return f

In [9]:
def getY(nbY, data_prefix):
    y = np.zeros(nbY)
    for yIdx in range( nbY ):
        filename = data_prefix + f"_000_{yIdx:03d}.amp"
        with open(filename) as file:
            for line in file:
                if "StepAxis=y" in line:
                    y[yIdx] = float(line.split(" ")[-2])
                elif "Data#1" in line:
                    break
    print(root_dir + " Ny = {}, ymin = {}, ymax = {}".format(y.size, np.amin(y), np.amax(y)))
    return y

In [10]:
f = getF( Nf, dat_011)
y = getY( Ny, dat_011)

/home/pleroy/DATA/DIADEM/bfu Nf = 9, fmin = 28.0, fmax = 36.0
/home/pleroy/DATA/DIADEM/bfu Ny = 81, ymin = 56.9, ymax = 432.1


# Get amplitudes and phases

In [12]:
def get_headerSize(filename):
    with open(filename) as file:
        for counter, line in enumerate(file):
            if "Data#1" in line:
                break
    return counter + 1

In [13]:
def get_x_val(data_prefix, fi, yi, ext):
    filename = data_prefix + f"_{fi:03d}_{yi:03d}" + ext
    headerSize = get_headerSize(filename)
    # read the data
    x, val = np.genfromtxt(filename, skip_header=headerSize, unpack=True)
    return x, val

def getAmpPhaArrays(Nx, Ny, fi, data_prefix):
    ampArray = np.zeros((Ny, Nx))
    phaArray = np.zeros((Ny, Nx))
    for yi in range(Ny):
        x, amp = get_x_val(data_prefix, fi, yi, ".amp")
        ampArray[yi, :] = amp[:]
        x, pha = get_x_val(data_prefix, fi, yi, ".pha")
        phaArray[yi, :] = pha[:]
    return ampArray, phaArray

In [14]:
fi = 0
yi = 0
x0, amp0 = get_x_val(dat_011, fi, yi, ".amp")
x0_setpoints = np.arange(23.4, 398.6 + 4.69, 4.69)
Nx = x0.size
# (left, right, bottom, top)
extent = [ x0[-1], x0[0], y[-1], y[0] ]
extent = [ x0[-1]-x0y0[0], x0[0]-x0y0[0], y[-1]-x0y0[1], y[0]-x0y0[1] ]
cmap='jet'

In [17]:
X0, Y = np.meshgrid(x0_setpoints, y)
x0.shape, y.shape, X0.shape, Y.shape

((81,), (81,), (81, 81), (81, 81))

In [18]:
fi = 4
print("get data for f = {}".format(f[fi]))

ampAndPhaArrays = {}
ampAndPhaArrays["011"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_011), dir_011, pre_011 )
ampAndPhaArrays["012"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_012), dir_012, pre_012 )
ampAndPhaArrays["013_0"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_013_0), dir_013_0, pre_013_0 )
ampAndPhaArrays["013_1"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_013_1), dir_013_1, pre_013_1 )
ampAndPhaArrays["014_0"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_014_0), dir_014_0, pre_014_0 )
ampAndPhaArrays["014_1"] = ( *getAmpPhaArrays(Nx, Ny, fi, dat_014_1), dir_014_1, pre_014_1 )

get data for f = 32.0


In [19]:
allAmpArrays = ( [ampAndPhaArrays[i][0] for i in ampAndPhaArrays.keys()] )
vmaxAll = np.amax(allAmpArrays)
vminAll = np.amin(allAmpArrays)
myVmax = 10
myVmin = -40
cmap='viridis'

In [54]:
fig, ((ax0,ax1), (ax2, ax3)) = plt.subplots(2,2)

im = ax0.pcolor(X0, Y, ampAndPhaArrays["011"][0], vmin=myVmin, vmax=myVmax, cmap=cmap )
dia.addColorBar(im, ax0)
ax0.set_title("pcolor")
ax0.set_xlabel("x")
ax0.set_ylabel("y")

im = ax2.imshow(ampAndPhaArrays["011"][0], vmin=myVmin, vmax=myVmax, cmap=cmap, origin='lower' )
dia.addColorBar(im, ax1)
ax2.set_title("imshow origin='lower'")
ax2.set_xlabel("xn")
ax2.set_ylabel("yn")

fig.suptitle("Ey " + dir_011)



Text(0.5,0.98,'Ey /20181015/011_3l_05l_+45')

In [57]:
fig, (ax0,ax1) = plt.subplots(1,2)

im = ax0.pcolor(X0, Y, ampAndPhaArrays["011"][0], vmin=myVmin, vmax=myVmax, cmap=cmap )
dia.addColorBar(im, ax0)
ax0.set_title("pcolor")
ax0.set_xlabel("x")
ax0.set_ylabel("y")

im = ax1.imshow(ampAndPhaArrays["011"][0], vmin=myVmin, vmax=myVmax, cmap=cmap, origin='lower' )
dia.addColorBar(im, ax1)
ax1.set_title("imshow origin='lower'")
ax1.set_xlabel("xn")
ax1.set_ylabel("yn")

fig.suptitle("Ey " + dir_011)

Text(0.5,0.98,'Ey /20181015/011_3l_05l_+45')

In [58]:
fig, (ax0,ax1) = plt.subplots(1,2)

im = ax0.pcolor(X0, Y, ampAndPhaArrays["012"][0], vmin=myVmin, vmax=myVmax, cmap=cmap )
dia.addColorBar(im, ax0)
ax0.set_title("pcolor")
ax0.set_xlabel("x")
ax0.set_ylabel("y")

im = ax1.imshow(ampAndPhaArrays["012"][0], vmin=myVmin, vmax=myVmax, cmap=cmap, origin='lower' )
dia.addColorBar(im, ax1)
ax1.set_title("imshow origin='lower'")
ax1.set_xlabel("xn")
ax1.set_ylabel("yn")

fig.suptitle("Ex " + dir_011)

Text(0.5,0.98,'Ex /20181015/011_3l_05l_+45')

In [60]:
fig, (ax0,ax1) = plt.subplots(1,2)

im = ax0.pcolor(X0, Y, np.rot90(ampAndPhaArrays["012"][0],k=-1), vmin=myVmin, vmax=myVmax, cmap=cmap )
dia.addColorBar(im, ax0)
ax0.set_title("pcolor")
ax0.set_xlabel("x")
ax0.set_ylabel("y")

im = ax1.imshow(np.rot90(ampAndPhaArrays["012"][0],k=-1), vmin=myVmin, vmax=myVmax, cmap=cmap, origin='lower' )
dia.addColorBar(im, ax1)
ax1.set_title("imshow origin='lower'")
ax1.set_xlabel("xn")
ax1.set_ylabel("yn")

fig.suptitle("Ex " + dir_011)

Text(0.5,0.98,'Ex /20181015/011_3l_05l_+45')

In [53]:
fig, (ax0,ax1) = plt.subplots(1,2)

im = ax0.pcolor(X0, Y, ampAndPhaArrays["012"][0], vmin=myVmin, vmax=myVmax, cmap=cmap )
dia.addColorBar(im, ax0)
ax0.set_title("pcolor")
ax0.set_xlabel("x")
ax0.set_ylabel("y")

im = ax1.imshow(ampAndPhaArrays["012"][0], vmin=myVmin, vmax=myVmax, cmap=cmap, origin='lower' )
dia.addColorBar(im, ax1)
ax1.set_title("imshow origin='lower'")
ax1.set_xlabel("xn")
ax1.set_ylabel("yn")

fig.suptitle("Ex " + dir_012)

Text(0.5,0.98,'Ex /20181015/011_3l_05l_+45/ey')

In [43]:
plt.figure()
plt.imshow(ampAndPhaArrays["011"][1], extent=extent)
title = "amplitudes"
plt.colorbar()

In [45]:
def addSubPlotAmp(sub, nb, extent, vmin, vmax, cmap, k=0):
    ax = plt.subplot(sub)
    if k == 0:
        ampArray = ampAndPhaArrays[nb][0]
    else:
        ampArray = np.rot90(ampAndPhaArrays[nb][0], k=k)
    im = ax.imshow(ampArray, extent=extent, vmin=vmin, vmax=vmax, cmap=cmap)
    title = "amplitude\n" + ampAndPhaArrays[nb][2] + "/" + ampAndPhaArrays[nb][3]
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad="5%")
    plt.colorbar(im, cax=cax)
    ax.set_title(title)
    ax.grid(color='k',linestyle='--')
    
def addSubPlotPha(sub, nb, cmap, k=0):
    plt.subplot(sub)
    if k == 0:
        ampArray = ampAndPhaArrays[nb][1]
    else:
        ampArray = np.rot90(ampAndPhaArrays[nb][1],k=k)
    plt.imshow(ampArray, cmap=cmap)
    title = "phase\n" + ampAndPhaArrays[nb][2] + "/" + ampAndPhaArrays[nb][3]
    plt.colorbar()
    plt.title(title)

In [46]:
plt.figure()

addSubPlotAmp(121, "011", extent, myVmin, myVmax, cmap)
addSubPlotAmp(122, "012", extent, myVmin, myVmax, cmap, -1)

In [47]:
plt.figure()

addSubPlotPha(121, "011", cmap)
addSubPlotPha(122, "012", cmap)

# Save data

In [ ]:
def saveData(root_dir, nb):
    header = "x y amp pha"
    filename = root_dir + ampAndPhaArrays[nb][2] + "_amp_pha_32GHz.data"
    print("save " + filename)
    np.savetxt(filename, 
               np.column_stack(( 
                   X0.flatten(), 
                   Y.flatten(), 
                   ampAndPhaArrays[nb][0].flatten(), 
                   ampAndPhaArrays[nb][1].flatten())),
               header=header,
               comments=""
              )

In [ ]:
saveData(root_dir, "011")
saveData(root_dir, "012")
saveData(root_dir, "013_0")
saveData(root_dir, "013_1")
saveData(root_dir, "014_0")
saveData(root_dir, "014_1")